Grupo de Pesquisa Ideologia e Análise de Discurso - UFPel (https://wp.ufpel.edu.br/idad/)

# **Minicurso:** *OpenAI Whisper - transcrição automática para pesquisa* (https://wp.ufpel.edu.br/idad/minicursos/)

Ministrante: Thales Morbach Lange (http://lattes.cnpq.br/3650755639394586)



---



## **OpenAI Whisper** (https://github.com/openai/whisper)

---



### 1 - Dependências e bibliotecas

In [ ]:
# Instalação de dependências
!pip install git+https://github.com/openai/whisper.git
!pip install torch torchaudio
!sudo apt update && sudo apt install ffmpeg
!pip install ipywidgets
!pip install google-colab

# Importações de bibliotecas
import os
import whisper
import ipywidgets as widgets
import numpy as np
from IPython.display import display, clear_output, Audio
from google.colab import drive, files, auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import datetime

print("✅ Bibliotecas instaladas e importadas com sucesso!")

### 2 - Modelo Whisper

In [ ]:
# Configuração do modelo
modelo = "medium"  # Pode ser: "tiny", "base", "small", "medium", "large"

# Carregar o modelo
print(f"⏳ Carregando modelo {modelo}...")
model = whisper.load_model(modelo)
print("✅ Modelo carregado e pronto para uso!")

### 3 - Idioma

In [ ]:
# Dicionário de idiomas suportados
WHISPER_LANGUAGES = {
    "Auto (Detectar automaticamente)": None,
    "Inglês": "en", "Português": "pt", "Espanhol": "es",
    "Francês": "fr", "Alemão": "de", "Italiano": "it",
    "Holandês": "nl", "Russo": "ru", "Japonês": "ja",
    "Chinês (Mandarim)": "zh", "Coreano": "ko",
    "Árabe": "ar", "Hindi": "hi"
}

# Widget de seleção
language_dropdown = widgets.Dropdown(
    options=list(WHISPER_LANGUAGES.keys()),
    value="Auto (Detectar automaticamente)",
    description='Idioma:',
    layout=widgets.Layout(width='80%')
)

# Função auxiliar
def get_selected_language():
    return WHISPER_LANGUAGES[language_dropdown.value]

# Exibição
print("\n🌍 Selecione o idioma para transcrição:")
display(language_dropdown)

### 4 - Transcrição

In [ ]:
# Célula 4: Transcrição

# Variáveis globais
global resultado_transcricao, saved_file_path
resultado_transcricao = None
saved_file_path = None

# Widgets principais
file_path_input = widgets.Text(
    value='',
    placeholder='Cole o caminho do arquivo (ex: /content/audio.mp3)',
    description='Arquivo:',
    layout=widgets.Layout(width='80%')
)

transcribe_button = widgets.Button(
    description='Transcrever',
    button_style='primary',
    layout=widgets.Layout(width='150px')
)

format_dropdown = widgets.Dropdown(
    options=['.txt', '.srt', '.vtt'],
    value='.txt',
    description='Formato:',
    layout=widgets.Layout(width='150px'),
    disabled=True
)

download_button = widgets.Button(
    description='Fazer Download',
    button_style='success',
    layout=widgets.Layout(width='150px'),
    disabled=True
)

drive_button = widgets.Button(
    description='Salvar no Drive',
    button_style='info',
    icon='google-drive',
    layout=widgets.Layout(width='150px'),
    disabled=True
)

# Widget para pasta do Drive
drive_folder_input = widgets.Text(
    value='',
    placeholder='Cole o caminho da pasta (ex: /content/drive/MyDrive/Pasta)',
    description='Pasta Drive:',
    layout=widgets.Layout(width='80%'),
    disabled=True,
    style={'description_width': 'initial'}
)

# Botão para navegar no Drive
drive_browse_button = widgets.Button(
    description='Procurar',
    button_style='',
    layout=widgets.Layout(width='100px'),
    disabled=True
)

# Containers dinâmicos
action_buttons_container = widgets.VBox([], layout=widgets.Layout(margin='10px 0 0 0'))
drive_options_container = widgets.VBox([], layout=widgets.Layout(margin='10px 0 0 0'))

# Área de saída principal
transcription_output = widgets.Output()

# Funções principais
def transcrever_audio(b):
    global resultado_transcricao

    with transcription_output:
        clear_output()
        caminho_audio = file_path_input.value.strip()

        if not os.path.exists(caminho_audio):
            print("❌ Arquivo não encontrado")
            return

        try:
            language = get_selected_language()
            print(f"🔊 Transcrevendo: {os.path.basename(caminho_audio)}...")
            resultado_transcricao = model.transcribe(
                caminho_audio,
                language=language,
                verbose=True
            )

            # Ativa controles após transcrição
            format_dropdown.disabled = False
            download_button.disabled = False
            drive_button.disabled = False
            drive_folder_input.disabled = False
            drive_browse_button.disabled = False

            # Mostra os controles dinamicamente após a mensagem
            print("\n✅ Transcrição concluída!")
            action_buttons_container.children = [
                widgets.HBox([format_dropdown, download_button, drive_button])
            ]
            drive_options_container.children = [
                widgets.HBox([drive_folder_input, drive_browse_button])
            ]

        except Exception as e:
            print(f"❌ Erro: {str(e)}")
            action_buttons_container.children = []
            drive_options_container.children = []

def handle_download(b):
    global saved_file_path

    if not resultado_transcricao:
        return

    nome_base = os.path.splitext(os.path.basename(file_path_input.value))[0] or "transcricao"
    output_path = f"/content/{nome_base}_transcricao{format_dropdown.value}"

    try:
        # Salvamento local
        if format_dropdown.value == '.txt':
            with open(output_path, 'w', encoding='utf-8') as f:
                # Adiciona cabeçalho em negrito (usando asteriscos para simular negrito)
                f.write(f"**{nome_base}**\n\n")
                for segment in resultado_transcricao['segments']:
                    f.write(segment['text'].strip() + '\n')

        elif format_dropdown.value == '.srt':
            with open(output_path, 'w', encoding='utf-8') as f:
                for i, segment in enumerate(resultado_transcricao['segments'], 1):
                    start = segment['start']
                    end = segment['end']
                    f.write(f"{i}\n{start:.2f} --> {end:.2f}\n{segment['text'].strip()}\n\n")

        # Download automático
        files.download(output_path)
        saved_file_path = output_path

    except Exception as e:
        with transcription_output: print(f"❌ Erro ao processar: {str(e)}")

def handle_drive_save(b):
    if not resultado_transcricao:
        return

    nome_base = os.path.splitext(os.path.basename(file_path_input.value))[0] or "transcricao"
    output_path = f"/content/{nome_base}_transcricao{format_dropdown.value}"

    try:
        # Primeiro salva localmente
        if format_dropdown.value == '.txt':
            with open(output_path, 'w', encoding='utf-8') as f:

                f.write(f"{nome_base}\n\n")
                for segment in resultado_transcricao['segments']:
                    f.write(segment['text'].strip() + '\n')

        elif format_dropdown.value == '.srt':
            with open(output_path, 'w', encoding='utf-8') as f:
                for i, segment in enumerate(resultado_transcricao['segments'], 1):
                    start = segment['start']
                    end = segment['end']
                    f.write(f"{i}\n{start:.2f} --> {end:.2f}\n{segment['text'].strip()}\n\n")

        # Verifica se o Drive está montado
        if not os.path.exists('/content/drive'):
            drive.mount('/content/drive')
            print("\n✅ Google Drive montado com sucesso!")

        # Determina o caminho de destino
        drive_path = drive_folder_input.value.strip() or '/content/drive/MyDrive/'

        # Garante que o caminho termina com /
        if not drive_path.endswith('/'):
            drive_path += '/'

        # Cria o diretório se não existir
        os.makedirs(drive_path, exist_ok=True)

        # Copia o arquivo para o Drive
        final_drive_path = os.path.join(drive_path, os.path.basename(output_path))
        os.system(f'cp "{output_path}" "{final_drive_path}"')

        with transcription_output:
            print(f"\n✅ Arquivo salvo no Drive:")
            print(f"📂 {final_drive_path}")
            print("\n⚠️ Se não encontrar o arquivo, verifique se:")
            print("- O caminho está correto")
            print("- O Drive está montado (caso não esteja, o script monta automaticamente)")

    except Exception as e:
        with transcription_output: print(f"❌ Erro ao salvar no Drive: {str(e)}")

def browse_drive(b):
    with transcription_output:
        clear_output()
        print("📂 Navegação no Google Drive")
        print("\nPara encontrar o caminho da pasta:")
        print("1. Abra o painel de arquivos (ícone de pasta à esquerda)")
        print("2. Navegue até a pasta desejada")
        print("3. Clique com o botão direito na pasta")
        print("4. Selecione 'Copiar caminho'")
        print("5. Cole no campo acima\n")
        print("Exemplo: /content/drive/MyDrive/MinhasTranscricoes")

# ========== CONFIGURAÇÃO ==========
transcribe_button.on_click(transcrever_audio)
download_button.on_click(handle_download)
drive_button.on_click(handle_drive_save)
drive_browse_button.on_click(browse_drive)

# Layout principal
display(widgets.VBox([
    widgets.HBox([file_path_input, transcribe_button]),
    transcription_output,
    action_buttons_container,  # Container para botões de ação
    drive_options_container    # Container para opções do Drive
]))

---

### 5 - Transcrição em lotes

In [1]:
# Célula 5: Transcrição em lotes no Google Drive

# Montar Google Drive
drive.mount('/content/drive')

# Criar os widgets da interface
header = widgets.HTML("<h2>Transcrição automática em lotes com Whisper</h2>")

info_text = widgets.HTML(
    "<p>Selecione a pasta de origem no Google Drive e a pasta de destino para as transcrições</p>"
)

drive_path_input = widgets.Text(
    value='',
    placeholder='Digite o caminho da pasta no Google Drive',
    description='Origem:',
    disabled=False,
    layout=widgets.Layout(width='80%')
)

dest_path_input = widgets.Text(
    value='',
    placeholder='Digite o caminho da pasta de destino',
    description='Destino:',
    disabled=False,
    layout=widgets.Layout(width='80%')
)

botao_listar = widgets.Button(
    description='Listar Arquivos',
    button_style='info',
    tooltip='Clique para listar os arquivos de áudio na pasta de origem'
)

botao_transcrever = widgets.Button(
    description='Iniciar Transcrição',
    button_style='success',
    tooltip='Clique para iniciar a transcrição'
)

progresso = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progresso:',
    bar_style='info',
    style={'bar_color': 'green'},
    orientation='horizontal'
)

file_count = widgets.Label(value="0 arquivos encontrados")
saida = widgets.Output()

# Função para listar arquivos
def listar_arquivos(b):
    with saida:
        clear_output()
        origem = drive_path_input.value.strip()

        if not origem:
            print("Por favor, informe o caminho de origem!")
            return

        if not os.path.exists(origem):
            print(f"A pasta {origem} não existe!")
            return

        arquivos = [f for f in os.listdir(origem)
                   if f.lower().endswith(('.mp3', '.wav', '.m4a', '.ogg', '.flac', '.mp4', '.mpeg'))]

        file_count.value = f"{len(arquivos)} arquivos encontrados"

        if arquivos:
            print("Arquivos encontrados:")
            for i, arquivo in enumerate(arquivos, 1):
                print(f"{i}. {arquivo}")
        else:
            print("Nenhum arquivo de áudio encontrado na pasta!")

# Função para transcrever
def iniciar_transcricao(b):
    with saida:
        clear_output()
        origem = drive_path_input.value.strip()
        destino = dest_path_input.value.strip()

        if not origem or not destino:
            print("Por favor, informe os caminhos de origem e destino!")
            return

        if not os.path.exists(origem):
            print(f"A pasta de origem {origem} não existe!")
            return

        # Criar pasta de destino se não existir
        os.makedirs(destino, exist_ok=True)

        arquivos = [f for f in os.listdir(origem)
                   if f.lower().endswith(('.mp3', '.wav', '.m4a', '.ogg', '.flac', '.mp4', '.mpeg'))]

        if not arquivos:
            print("Nenhum arquivo de áudio encontrado na pasta de origem!")
            return

        total_arquivos = len(arquivos)
        progresso.max = total_arquivos
        progresso.value = 0

        print(f"Iniciando transcrição de {total_arquivos} arquivos...")
        print(f"Origem: {origem}")
        print(f"Destino: {destino}\n")

        for i, arquivo in enumerate(arquivos, 1):
            caminho_origem = os.path.join(origem, arquivo)
            nome_base = os.path.splitext(arquivo)[0]
            caminho_destino = os.path.join(destino, f"{nome_base}_transcricao.txt")

            print(f"\nProcessando {i}/{total_arquivos}: {arquivo}...")

            try:
                # Transcrever
                resultado = model.transcribe(caminho_origem)

                # Salvar com cabeçalho em negrito
                with open(caminho_destino, "w", encoding="utf-8") as f:

                    f.write(f"{nome_base}\n\n")
                    for segment in resultado["segments"]:
                        f.write(segment["text"] + "\n")

                print(f"✔ Transcrição salva em: {caminho_destino}")
            except Exception as e:
                print(f"✖ Erro ao processar {arquivo}: {str(e)}")

            progresso.value = i

        print("\n✅ Processamento concluído!")

# Conectar os botões às funções
botao_listar.on_click(listar_arquivos)
botao_transcrever.on_click(iniciar_transcricao)

# Organizar a interface
box_caminhos = widgets.VBox([
    drive_path_input,
    dest_path_input
])

box_botoes = widgets.HBox([
    botao_listar,
    botao_transcrever
])

box_progresso = widgets.VBox([
    file_count,
    progresso
])

box_principal = widgets.VBox([
    header,
    info_text,
    box_caminhos,
    box_botoes,
    box_progresso,
    saida
])

# Exibir a interface
display(box_principal)

NameError: name 'drive' is not defined

---

## **Ferramenta extra: yt-dlp** (https://github.com/yt-dlp/yt-dlp)

In [ ]:
# yt-dlp - baixador de áudio/vídeo por linha de comando (https://github.com/yt-dlp/yt-dlp)

  # Instalação do yt-dlp
!pip install -U yt-dlp

    # Importar bibliotecas necessárias
import os
from yt_dlp import YoutubeDL

    # Configuração para exibir o progresso
class ProgressLogger:
    def debug(self, msg):
        pass

    def info(self, msg):
        pass

    def warning(self, msg):
        pass

    def error(self, msg):
        print(msg)

    # Função para baixar vídeo como MP3 com progresso
def baixar_mp3_com_progresso(url_video):
    # Configurações do yt-dlp
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'outtmpl': os.path.join('/content', '%(id)s.%(ext)s'),
        'progress_hooks': [hook_progresso],
        'logger': ProgressLogger(),
    }

    try:
        with YoutubeDL(ydl_opts) as ydl:
            print("Iniciando download...")
            info = ydl.extract_info(url_video, download=True)
            filename = ydl.prepare_filename(info).replace('.webm', '.mp3').replace('.m4a', '.mp3')
            print(f"\nDownload concluído: {filename}")
            return filename
    except Exception as e:
        print(f"\nErro ao baixar o vídeo: {e}")
        return None

# Função para mostrar o progresso
def hook_progresso(d):
    if d['status'] == 'downloading':
        percent = d.get('_percent_str', 'N/A')
        speed = d.get('_speed_str', 'N/A')
        eta = d.get('_eta_str', 'N/A')
        print(f"\rProgresso: {percent} | Velocidade: {speed} | Tempo restante: {eta}", end='', flush=True)
    elif d['status'] == 'finished':
        print("\nConvertendo para MP3...")

# Exemplo de uso
print("YT-DLP no Google Colab - Download para MP3")
print("----------------------------------------")
url = input("Cole a URL do vídeo que deseja baixar como MP3: ")
baixar_mp3_com_progresso(url)